## Visualizing Existing VPC Peers And Network Collisions
This notebook uses the AWS helper functions to gather data about VPCs across multiple profiles to show where there are existing peering networks and where there are collisions between networks. The calculated data is displayed using the [Bokeh Visualization Library](https://bokeh.org/).

In [ ]:
%run -i helper/aws.py
%run -i helper/aws_functions.py
aws_ctx = await aws_connect(
    profiles=["sandbox", "prod"], regions=AWS_ALL_REGIONS, svc_include=["ec2"]
)

import math
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
from copy import deepcopy
from itertools import cycle

import bokeh
import bokeh.colors
import pandas as pd
from bokeh.models import (
    ColumnDataSource,
    EdgesAndLinkedNodes,
    GraphRenderer,
    MultiLine,
    NodesAndLinkedEdges,
    Scatter,
    StaticLayoutProvider,
)
from bokeh.palettes import Colorblind8, Spectral11
from bokeh.plotting import figure, output_notebook, show
from ipaddr import IPNetwork
from perspective import PerspectiveWidget

pd.set_option("display.max_rows", 10000)
pd.set_option("display.max_columns", 100)
output_notebook()


def get_data_for_vpc(ctx_obj, acct, region):
    instances = [
        instance
        for reservation in retrieve_all_pages(
            ctx_obj[acct][region]["ec2"], "describe_instances"
        ).get("Reservations")
        for instance in reservation.get("Instances")
    ]
    security_groups = retrieve_all_pages(
        ctx_obj[acct][region]["ec2"], "describe_security_groups"
    ).get("SecurityGroups")
    vpcs = retrieve_all_pages(ctx_obj[acct][region]["ec2"], "describe_vpcs").get("Vpcs")
    vpc_peers = retrieve_all_pages(
        ctx_obj[acct][region]["ec2"], "describe_vpc_peering_connections"
    ).get("VpcPeeringConnections")
    subnets = retrieve_all_pages(ctx_obj[acct][region]["ec2"], "describe_subnets").get(
        "Subnets"
    )

    return dict(
        instances=instances,
        security_groups=security_groups,
        vpcs=vpcs,
        vpc_peers=vpc_peers,
        subnets=subnets,
    )

In [ ]:
data = perform_aws_operations(aws_ctx, get_data_for_vpc)

In [ ]:
[(acct, region) for acct, reg_ctx in aws_ctx.items() for region, ctx in reg_ctx.items()]

## Answer the following questions:
- Which VPCs have how many active Ec2 instances?
- Which VPC CIDRs conflict or overlap

In [ ]:
all_active_instances = [
    dict(acct=acct, region=region, **item)
    for acct, reg_ctx in data.items()
    for region, sections in reg_ctx.items()
    for section, data_items in sections.items()
    if section == "instances"
    for item in data_items
    if item.get("State", {}).get("Code") in [16, 80]
]

all_vpc_peers = [
    dict(acct=acct, region=region, **item)
    for acct, reg_ctx in data.items()
    for region, sections in reg_ctx.items()
    for section, data_items in sections.items()
    if section == "vpc_peers"
    for item in data_items
    if item.get("Status", {}).get("Code") == "active"
]

reduced_peers = defaultdict(list)
vpc_peer_id_pairs_unique = sorted(
    set(
        [
            (
                peer.get("AccepterVpcInfo").get("VpcId"),
                peer.get("RequesterVpcInfo").get("VpcId"),
            )
            for peer in all_vpc_peers
        ]
    )
)
vpc_peer_id_pairs_reversed = [(pair[1], pair[0]) for pair in vpc_peer_id_pairs_unique]
vpc_peer_id_pairs_all = vpc_peer_id_pairs_unique + vpc_peer_id_pairs_reversed

for pair in vpc_peer_id_pairs_all:
    reduced_peers[pair[0]].append(pair[1])

# Calculate collisions
table_data = [
    dict(id=item.get("VpcId"), acct=acct, region=region, cidr=item.get("CidrBlock"))
    for acct, reg_ctx in data.items()
    for region, sections in reg_ctx.items()
    for section, data_items in sections.items()
    if section == "vpcs"
    for item in data_items
]
table_copy = deepcopy(table_data)
for network in table_data:
    network.update(
        dict(
            collisions=[
                n.get("id")
                for n in table_copy
                if n != network
                and IPNetwork(n.get("cidr")).Contains(IPNetwork(network.get("cidr")))
            ],
            peers=reduced_peers.get(network.get("id"), []),
            active_instance_count=len(
                [i for i in all_active_instances if i.get("VpcId") == network.get("id")]
            ),
        )
    )
active_only_table_data = [n for n in table_data if n.get("active_instance_count") > 0]

collision_pairs = [
    (network.get("id"), collision)
    for network in active_only_table_data
    for collision in network.get("collisions")
]

## Prepare Graph Layout and Show final data

In [ ]:
color_wheel = cycle(Spectral11 + Colorblind8)

data_frame = pd.DataFrame(active_only_table_data)

total_node_count = len(data_frame)
enlarge_factor = total_node_count * 2.95
circ = [i * 2 * math.pi / total_node_count for i in range(total_node_count)]
x = [math.cos(i) * enlarge_factor for i in circ]
y = [math.sin(i) * enlarge_factor for i in circ]
data_frame["index"] = data_frame["id"]
data_frame.set_index("id", inplace=True)

data_frame["x"] = x
data_frame["y"] = y

data_frame["size"] = data_frame["active_instance_count"].apply(
    lambda x: 10 + (5 * math.log1p(x))
)
data_frame["node_color"] = data_frame.apply(lambda _: next(color_wheel), axis=1)

PerspectiveWidget(data_frame)

## Graphing Existing Peers

In [ ]:
data_source = ColumnDataSource(data_frame)

hover_tips = [
    ("vpc_id", "@index"),
    ("account", "@acct"),
    ("region", "@region"),
    ("CIDR", "@cidr"),
    ("instances", "@active_instance_count"),
]
p = figure(
    plot_width=960,
    plot_height=960,
    toolbar_location="below",
    tooltips=hover_tips,
    x_range=(-140.0, 140.0),
    y_range=(-140.0, 140.0),
)

# Turn off x and y axis labels and ticks...
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_size = '0pt'
p.yaxis.major_label_text_font_size = '0pt'

graph = GraphRenderer()
graph.node_renderer.data_source.data = dict(data_source.data)
graph.node_renderer.glyph = Scatter(
    marker="square", x="x", y="y", size="size", fill_color="node_color", fill_alpha=0.85
)
graph.node_renderer.selection_glyph = Scatter(
    marker="square", fill_alpha=0.85
)  # x="x", y="y",
graph.node_renderer.hover_glyph = Scatter(marker="square", fill_alpha=0.75)

graph.edge_renderer.data_source.data = dict(
    start=[i[0] for i in vpc_peer_id_pairs_unique],
    end=[i[1] for i in vpc_peer_id_pairs_unique],
)
graph.edge_renderer.glyph = MultiLine(
    line_color="#CCCCCC", line_alpha=0.5, line_width=5
)
graph.edge_renderer.selection_glyph = MultiLine(line_color="#777777", line_width=5)
graph.edge_renderer.hover_glyph = MultiLine(line_color="#7777FF", line_width=5)

graph_layout = {id: (item["x"], item["y"]) for id, item in data_frame.iterrows()}
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)
graph.selection_policy = NodesAndLinkedEdges()
# graph.inspection_policy = EdgesAndLinkedNodes()
p.renderers.append(graph)

show(p)

## Graphing Network Collisions

In [ ]:
data_source = ColumnDataSource(data_frame)

hover_tips = [
    ("vpc_id", "@index"),
    ("account", "@acct"),
    ("region", "@region"),
    ("CIDR", "@cidr"),
    ("instances", "@active_instance_count"),
]
p = figure(
    plot_width=960,
    plot_height=960,
    toolbar_location="below",
    tooltips=hover_tips,
    x_range=(-140.0, 140.0),
    y_range=(-140.0, 140.0),
)
# Turn off x and y axis labels and ticks...
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.xaxis.major_label_text_font_size = '0pt'
p.yaxis.major_label_text_font_size = '0pt'

graph = GraphRenderer()
graph.node_renderer.data_source.data = dict(data_source.data)

graph.node_renderer.glyph = Scatter(
    marker="square", x="x", y="y", size="size", fill_color="node_color", fill_alpha=0.99
)
graph.node_renderer.selection_glyph = Scatter(
    marker="square", fill_alpha=0.85
)  # x="x", y="y",
graph.node_renderer.hover_glyph = Scatter(marker="square", fill_alpha=0.75)

graph.edge_renderer.data_source.data = dict(
    start=[collision_pair[0] for collision_pair in collision_pairs],
    end=[collision_pair[1] for collision_pair in collision_pairs],
)
graph.edge_renderer.glyph = MultiLine(
    line_color="#FF0000", line_alpha=0.65, line_width=5
)
graph.edge_renderer.selection_glyph = MultiLine(line_color="#777777", line_width=5)
graph.edge_renderer.hover_glyph = MultiLine(line_color="#7777FF", line_width=5)

graph_layout = {id: (item["x"], item["y"]) for id, item in data_frame.iterrows()}
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)
graph.selection_policy = NodesAndLinkedEdges()
# graph.inspection_policy = EdgesAndLinkedNodes()
p.renderers.append(graph)

show(p)